### Article 1
- 모델 적용

In [18]:
!ls

03_chatbot (1).ipynb     article                  datas
04_article.ipynb         article_classifier.ipynb


In [19]:
!rm -rf article
!mkdir -p article/models
!mkdir -p article/templates
!touch article/article.py
!touch article/templates/index.html
!cp model.pkl article/models/
!tree article

cp: model.pkl: No such file or directory
article
├── article.py
├── models
└── templates
    └── index.html

2 directories, 2 files


In [20]:
# route

In [21]:
%%writefile article/article.py
import pickle
import numpy as np
from flask import *
app = Flask(__name__)
app.config.update(
    TEMPLATES_AUTO_RELOAD=True, DEBUG=True
)
@app.route("/")
def index():
    return render_template("index.html")

app.run()

Overwriting article/article.py


In [22]:
# %%writefile article/templates/index.html
# <!DOCTYPE html>
# <html>
# <head>
#     <meta charset="utf-8"><title>DSS16</title>
#     <link href='https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.6.0/css/bootstrap-grid.min.css'>
# </head>
# <body>
#     <div class='container'>
#         <div class="row">
#             <input type='text' class="sentence", placegolder='문장을 입력하세요'>
#             <button type='button' class="prediction">

In [54]:
%%writefile article/templates/index.html
<!DOCTYPE html>
<html lang="en" dir="ltr">
    <head>
        <meta charset="utf-8">
        <title>데사스</title>
        <link rel="shortcut icon" href="https://asset.fastcampus.co.kr/favicon/favicon-16x16.png"  />
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.5.0/css/bootstrap.min.css">
        <style media="screen">
            container, .highcharts-container{
                margin: 0 auto;
            }
        </style>
    </head>
    <body>
        <div class="container pt-5">

            <!-- 입력창 -->
            <div class="row">
                <input class="form-control col-9 sentence" type="text" placeholder="type your sentence." 
                value="data science is fun hahaha.">
                <button type="button" class="btn btn-danger col-3 prediction">Prediction</button>
            </div>

            <!-- 결과창 -->
            <div class="row">
                <div class="col-15 alert alert-danger mt-2" role="alert">
                    A simple danger alert—check it out!
                </div>
            </div>

            <!-- 차트 -->
            <div class="row">
                <div id="container" class="w-100"></div>
            </div>

        </div>

        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
        <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/6.1.1/highcharts.js"></script>
        <script type="text/javascript">
        
            $(document).ready(function(){
                $('.prediction').click(function(){
                    var sentence = $('.sentence').val();
                    console.log(sentence);
                    var url = "/predic?sentence=" + sentence;
                    console.log(url);
                    $.getJSON(url, function(data){
                        console.log(data);

                        $('.alert').empty();
                        for(var i=0; i<data.category.length; i++){
                            var tag = "<p>" + data.category[i] + " : " + data.result[i] + "%</p>";
                            $('.alert').append(tag);
                        }

                        var series = [];
                        for(var i=0; i<data.category.length; i++){
                            series.push({
                                name: data.category[i],
                                y: data.result[i]
                            })
                        }
                        console.log(series);

                        draw_chart(series);

                    })

                })
            })

            function draw_chart(series){

                Highcharts.chart('container', {
                    chart: {
                        plotBackgroundColor: null,
                        plotBorderWidth: null,
                        plotShadow: false,
                        type: 'pie'
                    },
                    title: {
                        text: '데이터 사이언스 스쿨'
                    },
                    tooltip: {
                        pointFormat: '{series.name}: <b>{point.percentage:.1f}%</b>'
                    },
                    plotOptions: {
                        pie: {
                            allowPointSelect: true,
                            cursor: 'pointer',
                            dataLabels: {
                                enabled: false
                            },
                            showInLegend: true
                        }
                    },
                    series: [{
                        name: 'Brands',
                        colorByPoint: true,
                        data: series
                    }]
                });

            }
        </script>
    </body>
</html>

Overwriting article/templates/index.html


In [53]:
%%writefile article/article.py
from flask import *
import pickle
import os
import numpy as np
 
app = Flask(__name__)
app.config.update(TEMPLATES_AUTO_RELOAD=True,DEBUG=True)

with open('models/model.pkl',"rb") as file:
    model= pickle.load(file)

models = {}

def init():
    with open("models/model.pkl", "rb") as f:
        models["category"] = pickle.load(f)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predic")
def predic():
    result = {}
    result["category"] = ["politics","economics","social","life and culture","world","IT/science"]
    model = models["category"]
    
    sentence = request.values.get("sentence") 
    result["sentence"] = sentence

    result["result"] = list(np.round_(model.predict_proba([sentence])[0] * 100, 2))
    
    return jsonify(result)
    
#     response = jsonify(result)
#     response.headers.add('Access-Control-Allow-Origin', '*')
    
#      return response
init()
app.run()

Overwriting article/article.py


In [33]:
!tree article/

article/
├── article.py
├── model.pkl
├── models
│   └── model.pkl
└── templates
    └── index.html

2 directories, 4 files
